### 파이썬 추천 시스템 패키지 - Surprise

#### Surprise를 이용한 추천 시스템 구축

In [1]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [2]:
data = Dataset.load_builtin('ml-100k')
# 수행 시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

In [3]:
algo = SVD()
algo.fit(trainset)

In [4]:
predictions = algo.test(testset)
print('prediction type : ', type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type :  <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5830687706441164, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.7122655519012655, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.1130243256386905, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.517036522653651, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.5591381650381955, details={'was_impossible': False})]

In [5]:
[(pred.uid,pred.iid,pred.est) for pred in predictions[:3]]

[('120', '282', 3.5830687706441164),
 ('882', '291', 3.7122655519012655),
 ('535', '507', 4.1130243256386905)]

In [6]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.18   {'was_impossible': False}


In [7]:
accuracy.rmse(predictions)

RMSE: 0.9477


0.9477074500458136

#### Surprise 주요 모델 소개

In [8]:
import pandas as pd 

ratings = pd.read_csv('../data/ml-latest-small/ratings.csv')
# ratings_noh.csv 파일로 언로드 시 인덱스와 헤더를 모두 제거한 새로운 파일 생성
ratings.to_csv('../data/ml-latest-small/ratings_noh.csv', index=False, header=False)

In [9]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',',rating_scale=(0.5,5))
data=Dataset.load_from_file('../data/ml-latest-small/ratings_noh.csv', reader=reader)

In [10]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행 시마다 동일한 결과를 도출하기 위해 random_state 설정
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습하고 나서 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [12]:
import pandas as pd 
from surprise import Reader, Dataset

ratings = pd.read_csv('../data/ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))

# ratings DataFrame에서 칼럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 합니다.
data = Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)
traineset, testset = train_test_split(data,test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 1.0388


1.0387858758781252

#### 교차 검증과 하이퍼 파라미터 튜닝

In [13]:
from surprise.model_selection import cross_validate

# 판단스 DataFrame에서 Surprise 데이터 세트로 데이터 로딩
ratings = pd.read_csv('../data/ml-latest-small/ratings.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo,data, measures=['RMSE','MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8771  0.8764  0.8713  0.8719  0.8743  0.8742  0.0023  
MAE (testset)     0.6727  0.6715  0.6689  0.6705  0.6723  0.6712  0.0013  
Fit time          4.59    4.87    4.81    4.47    4.51    4.65    0.16    
Test time         0.14    0.25    0.13    0.14    0.26    0.18    0.06    


{'test_rmse': array([0.87710258, 0.87642213, 0.87128594, 0.8718801 , 0.87432179]),
 'test_mae': array([0.67266598, 0.67150014, 0.66890667, 0.67053324, 0.67225606]),
 'fit_time': (4.591114044189453,
  4.868176221847534,
  4.808330059051514,
  4.47021484375,
  4.509926795959473),
 'test_time': (0.14210844039916992,
  0.2490384578704834,
  0.13180279731750488,
  0.13795232772827148,
  0.2581772804260254)}

In [14]:
from surprise.model_selection import GridSearchCV

# 치ㅗ적화할 파라미터를 딕셔너리 현태로 지정
param_grid = {'n_epochs':[20,40,60],'n_factors':[50,100,200]}

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse','mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8772161211011879
{'n_epochs': 20, 'n_factors': 50}


#### Surprise를 이용한 개인화 영화추천 시스템 구축

In [15]:
# 아래 코드는 train_test_split( )으로 분리되지 않는 Dataset에 fit( )을 호출하여 오류를 발생합니다.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

AttributeError: 'DatasetAutoFolds' object has no attribute 'global_mean'

In [16]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성
data_folds = DatasetAutoFolds(ratings_file='../data/ml-latest-small/ratings_noh.csv', reader=reader)

# 전체 데이터를 학습 데이터로 생성함
trainset = data_folds.build_full_trainset()

In [17]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [18]:
# 영화에 대한 상세 속성 정보 DataFrame 로딩
movies = pd.read_csv('../data/ml-latest-small/movies.csv')

# userId=9의 movieId 데이터를 추출해 movieId=42 데이터가 있는지 확인
movieIds = ratings[ratings['userId']==9]['movieId']

if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [19]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [21]:
def get_unseen_surprise(ratings, movies, userId):
    # 입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()

    # 모든 영화의 movieId를 리스트로 생성
    total_movies = movies['movieId'].tolist()

    # 모든 영화의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후 리스트로 생성
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화 수 :', len(seen_movies) ,'추천 대상 영화 수 : ', len(unseen_movies), '정체 영화 수 : ', len(total_movies))

    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수 : 46 추천 대상 영화 수 :  9696 정체 영화 수 :  9742


In [22]:
def recomm_movie_by_surprise(algo, userId, unseen_movis, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음
    # [Prediction(uid='9', iid='i', est=3.69), Prediction(uid='9', iid='2'. est=2.98'),,,,,]

    # 이름 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행
    def sortkey_est(pred):
        return pred.est

    # sortkey_est() 반환값의 내림 차순으로 정렬 수행하고 top_n의 최상휘 값 추출
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions = predictions[:top_n]

    # top_n으로 추출된 영화의 정보 추추. 영화 아이디, 추천 예상 평점, 제목추출
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_rating = [pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [(id,title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]

    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo,9,unseen_movies,top_n=10)

print('##### Top~10 추천 영화 리스트 #####')
for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화 수 : 46 추천 대상 영화 수 :  9696 정체 영화 수 :  9742
##### Top~10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
